In [ ]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags

In [ ]:
df = pd.read_feather("C:\\Users\\jaspa\\Grant ML\\slp\\data\\all_game_data_df.feather")

In [ ]:
df.head()
df.columns

In [ ]:
path_list = df['player_1_frame_data_path'].to_list() + df['player_2_frame_data_path'].to_list()
print(path_list)

In [ ]:
# frame_data_df = pd.read_feather(path_list[0])

In [ ]:
# frame_data_df.columns
# frame_data_df['pre_buttons_physical'][2]

In [ ]:
def create_directories(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
# def process_file(path,time_1,time_2,time_3):
#     save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    
#     time_1_start = time.time()
#     frame_data_df = pd.read_feather(path)
#     time_1.append(time.time()-time_1_start)
    
#     time_2_start = time.time()
#     inputs = np.empty((4,frame_data_df.shape[0]),dtype = float)
#     for i in range(frame_data_df.shape[0]):
#         inputs[0, i] = frame_data_df['pre_joystick_x'][i]
#         inputs[1, i] = frame_data_df['pre_joystick_y'][i]
#         inputs[2, i] = frame_data_df['pre_cstick_x'][i]
#         inputs[3, i] = frame_data_df['pre_cstick_y'][i] 
#         # inputs[4:,i] = one_hot_encode_flags(slp.event.Buttons.Physical,frame_data_df['pre_buttons_physical'][i])
#     time_2.append(time.time()-time_2_start)
    
#     time_3_start = time.time()
#     full_path = os.path.join(save_path, str(uuid.uuid4()) + '.npy.gz')
#     with gzip.open(full_path, 'wb') as f:
#         np.save(f, inputs)
#     time_3.append(time.time() - time_3_start)
#     return inputs         


def process_file(path,time_1,time_2,time_3):
    save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    time_1_start = time.time()
    frame_data_df = pd.read_parquet(path,engine='pyarrow')
    time_1.append(time.time()-time_1_start)
    num_rows = frame_data_df.shape[0]

    # Assuming one_hot_encode_flags can process a series and return a 2D array
    # encoded_flags = one_hot_encode_flags(slp.event.Buttons.Physical, frame_data_df['pre_buttons_physical'])


    time_2_start = time.time()

    inputs = np.empty((4, num_rows), dtype=float)
    inputs[0, :] = frame_data_df['pre_joystick_x']
    inputs[1, :] = frame_data_df['pre_joystick_y']
    inputs[2, :] = frame_data_df['pre_cstick_x']
    inputs[3, :] = frame_data_df['pre_cstick_y']
    # inputs[4:, :] = encoded_flags
    time_2.append(time.time()-time_2_start)
    
    time_3_start = time.time()
    # full_path = os.path.join(save_path, f"{uuid.uuid4()}.npy.gz")
    # with gzip.open(full_path, 'wb') as f:
    #     np.save(f, inputs)
    time_3.append(time.time() - time_3_start)
    
    return inputs

In [ ]:
# process_file(path_list[0],1)
manager = Manager()
time_1 = manager.list()
time_2 = manager.list()
time_3 = manager.list()
        
# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(process_file)(path,time_1,time_2,time_3) for path in tqdm.tqdm(path_list))



In [ ]:
print(sum(time_1))
print(sum(time_2))
print(sum(time_3))
